In [1]:
%pip install pymilvus openai transformers torch

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
prompt_text = """
[배경 정보]

- 분석 대상 기업: 셀트리온
- 현재 시점: 2025년 1월 1일

[목표]

셀트리온의 24년 4분기 실적 분석 및 향후 전망에 대한 종합적인 기업 분석 보고서를 생성하는 것입니다.

[보고서 작성 가이드라인]

- 정보 출처 명확화: 보고서의 모든 내용은 오직 RAG를 통해 제공된 컨텍스트에만 근거해야 합니다. 컨텍스트에 명시적으로 언급되지 않은 외부 정보, 추측, 또는 개인적인 의견을 포함하지 마십시오.
- 객관성 유지: 사실에 기반하여 객관적이고 중립적인 톤으로 서술하십시오.
- 구조화: 아래 제시된 구조에 따라 정보를 논리적으로 구성하여 보고서를 작성하십시오.

[생성할 보고서의 구조 및 포함 내용 지침]

다음 구조를 따라 보고서를 작성하되, 각 섹션에 해당하는 내용을 제공된 컨텍스트(공시, 뉴스 기사)에서 찾아서 요약하고 기술하십시오.

1. 보고서 요약 (Executive Summary):
    - 컨텍스트에 기반한 셀트리온 4Q24 실적의 주요 특징 요약.
    - 컨텍스트에서 파악된 향후 사업 방향 또는 전망에 대한 핵심 내용 요약.
2. 2024년 4분기 실적 분석:
    - 실적 요인 분석: 공시 내용이나 뉴스 기사에서 언급된 4Q24 실적의 주요 변동 요인(긍정적/부정적 요인 모두)을 설명. (예: 특정 제품의 판매 호조/부진, 비용 증가/감소 요인, 일회성 손익 발생 등 공식적으로 언급된 내용)
3. 주요 사업 및 제품 동향:
    - 제품 관련 소식: 컨텍스트에서 언급된 주요 제품(예: 램시마SC, 유플라이마, 베그젤마, 짐펜트라, 스테키마 등) 관련 최신 동향(예: 주요 시장 출시/허가 현황, 판매 관련 언급, 생산 관련 소식 등)을 요약.
    - R&D 및 파이프라인: 컨텍스트에서 찾을 수 있는 신약 개발 진행 상황, 임상 결과 발표, 기술 도입 등 R&D 관련 중요 업데이트 사항을 기술.
    - CMO 사업: 위탁생산(CMO) 관련 계약, 생산 등 컨텍스트 내 관련 정보를 요약.
    - 기타 사업: 합병 관련 진행 상황 및 시너지 창출 노력 등 컨텍스트 내 기타 중요 사업 내용을 포함.
4. 시장 환경 및 전략 방향:
    - 주요 시장 활동: 컨텍스트에 나타난 주요 시장(예: 미국, 유럽)에서의 활동 내용(예: 신제품 출시, 허가 신청, 마케팅 활동, 시장 경쟁 관련 언급 등)을 요약.
    - 회사의 공식 전략: 컨텍스트의 공시나 보도자료 등에서 발표된 회사의 주요 경영 전략, 투자 계획, 파트너십 체결 등 내용을 정리.
5. 향후 전망 (공식 발표 기반):
    - 회사의 공식 입장/계획: 컨텍스트에서 확인되는 2025년 사업 계획, 목표, 신제품 출시 예정, 성장 전략 등 회사에서 공식적으로 발표한 향후 전망 관련 내용을 요약. (애널리스트의 예측이 아닌, 회사의 발표 내용 중심)
    - 미래 성과 영향 요인: 컨텍스트 정보를 바탕으로 향후 실적에 영향을 미칠 수 있는 주요 요인(예: 신제품 성과 기대, 진행 중인 R&D 중요성, 시장 환경 변화 등 공식 발표나 뉴스에서 강조된 내용)을 정리.
6. 기타 참고사항:
    - 컨텍스트(공시, 뉴스 기사)에서 언급된 기타 중요 정보, 잠재적 위험 요인 또는 기회 요인 등을 객관적으로 요약. (투자 추천이나 가치 판단은 제외)

[보고서 평가 기준]

평가 목표: 생성된 보고서가 주어진 컨텍스트(셀트리온 4Q24 공시 자료 및 관련 뉴스 기사)의 정보를 얼마나 정확하고 충실하게, 그리고 구조적으로 잘 요약했는지 평가합니다.

종합 평가 기준 (모든 목차 공통 적용):

1. 컨텍스트 충실성 (Context Fidelity): 보고서의 모든 내용이 오직 제공된 컨텍스트 정보에만 기반하는가? 외부 정보나 환각(Hallucination)은 없는가? (가장 중요)
2. 구조 준수성 (Structural Adherence): 제시된 6가지 목차 구조를 정확히 따르고 있는가?
3. 객관성 및 톤 (Objectivity & Tone): 보고서 전체적으로 객관적이고 사실 기반의 중립적인 톤을 유지하는가? 추측이나 주관적 평가는 배제되었는가?
4. 명확성 및 가독성 (Clarity & Readability): 사용된 언어가 명확하고 이해하기 쉬운가? 정보가 각 목차 내에서 논리적으로 구성되어 있는가?

목차별 세부 평가 기준:

1. 보고서 요약 (Executive Summary)

- 핵심 내용 반영도: 보고서 본문(2~6번 목차)의 핵심 내용(4Q24 실적 주요 특징, 향후 전망 핵심)을 정확하게 요약하고 있는가?
- 정확성 및 일관성: 요약된 내용이 컨텍스트 및 보고서 본문의 내용과 일치하며 왜곡이 없는가?
- 간결성: 핵심 내용을 간결하게 전달하는가? 불필요하게 상세하지 않은가?
- 포괄성: 실적 측면과 전망 측면을 균형 있게 포함하는가?

2. 2024년 4분기 실적 분석 (4Q24 Performance Review)

- 실적 요인 분석 정확성: 실적 변동 요인(매출 동인, 비용 요인 등) 설명이 컨텍스트(공시, 뉴스) 내용과 정확히 일치하는가?
- 실적 요인 분석 완전성: 컨텍스트에서 언급된 *중요한* 실적 변동 요인을 충분히 다루고 있는가?
- 컨텍스트 기반: 분석 내용이 컨텍스트 정보에만 근거하며 외부 해석이 배제되었는가?

3. 주요 사업 및 제품 동향 (Business & Product Developments)

- 정보 정확성: 제품 동향, R&D 업데이트, CMO 관련 기술 내용이 컨텍스트 정보와 사실적으로 일치하는가?
- 정보 완전성: 컨텍스트에서 언급된 주요 제품, R&D, CMO 관련 *중요* 업데이트 사항을 누락 없이 포함했는가?
- 관련성: 보고서 목차의 주제(사업 및 제품 동향)와 관련된 내용을 충실히 담고 있는가?
- 컨텍스트 기반: 내용이 컨텍스트(공시, 뉴스)에서 직접 확인 가능한 정보인가?

4. 시장 환경 및 전략 방향 (Market Environment & Strategy)

- 정보 정확성: 주요 시장(미국, 유럽 등) 활동 및 회사 전략(합병 시너지, 파트너십 등)에 대한 설명이 컨텍스트 정보와 일치하는가?
- 정보 완전성: 컨텍스트에서 강조된 주요 시장 활동 및 전략 방향을 충분히 포함하고 있는가?
- 관련성: 내용이 시장 환경 및 회사의 전략적 움직임에 초점을 맞추고 있는가?
- 컨텍스트 기반: 설명이 컨텍스트에서 제공된 정보의 범위를 벗어나지 않는가?

5. 향후 전망 (공식 발표 기반) (Future Outlook - Based on Official Statements)

- 공식 입장 정확성: 회사의 공식적인 향후 계획, 목표, 신제품 출시 일정 등을 컨텍스트 내용 그대로 정확하게 전달하는가?
- 공식 입장 완전성: 컨텍스트에서 언급된 회사의 *주요* 공식 전망 내용을 포함하고 있는가?
- 출처 명확성: 해당 내용이 회사의 '공식 발표'에 기반한 것임을 명확히 인지할 수 있게 서술되었는가? (추측성 서술과 구분되는가?)
- 컨텍스트 기반: 회사의 공식 발표 내용을 왜곡하거나 과장하지 않았는가?

6. 기타 참고사항 (Key Considerations - Based on Public Info)

- 정보 정확성: 언급된 참고사항(리스크, 기회 등)이 컨텍스트 정보에 사실적으로 기반하는가?
- 정보 완전성: 컨텍스트 내 다른 목차에 포함되지 않은 *중요한* 기타 정보, 리스크, 기회 요인을 포함하는가?
- 객관성: 투자 추천이나 주관적 가치 판단 없이, 사실 정보를 객관적으로 전달하는가?
- 관련성: 내용이 '기타 참고사항'으로서의 관련성을 가지는가?
- 컨텍스트 기반: 내용이 컨텍스트에서 직접 확인 가능한 정보인가?
"""

In [ ]:
import os
import torch
from transformers import AutoTokenizer, AutoModel
from pymilvus import connections, Collection, utility, FieldSchema, CollectionSchema, DataType
from openai import OpenAI
from dotenv import load_dotenv

import time

# --- 1. 설정값 ---

# .env 파일에서 환경 변수 로드
load_dotenv()

# OpenAI 설정
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    print("오류: OPENAI_API_KEY 환경 변수를 설정해주세요.")
    exit()
LLM_MODEL = "gpt-4o-mini" # 사용할 OpenAI 모델

# 임베딩 모델 설정 (Milvus 저장 시 사용했던 모델과 동일해야 함)
EMBEDDING_MODEL_NAME = "klue/bert-base"
VECTOR_DIM = 768

# Milvus 설정
MILVUS_HOST = os.getenv("MILVUS_HOST", "localhost")
MILVUS_PORT = os.getenv("MILVUS_PORT", "19530")
# COLLECTION_NAME = "celltrion_embedding" # <<<< 검색할 컬렉션 이름
# 만약 여러 컬렉션을 검색해야 한다면 리스트로 관리
COLLECTION_NAMES = ["celltrion_embeddings", "news_embeddings"]

# 검색 설정
SEARCH_TOP_K = 20 # Milvus에서 검색할 상위 K개 결과
SEARCH_PARAMS = {
    "metric_type": "L2",
    "params": {"nprobe": 10} # IVF_FLAT 인덱스 사용 시 nprobe 값 조절
    # "params": {"ef": 64} # HNSW 인덱스 사용 시 ef 값 조절
}

# --- 2. 모델 및 토크나이저 로드 ---
print("Loading embedding model and tokenizer...")
try:
    tokenizer = AutoTokenizer.from_pretrained(EMBEDDING_MODEL_NAME)
    embedding_model = AutoModel.from_pretrained(EMBEDDING_MODEL_NAME)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    embedding_model.to(device)
    embedding_model.eval() # 평가 모드
    print(f"Using device for embedding: {device}")
except Exception as e:
    print(f"Error loading embedding model: {e}")
    exit()

# --- 3. 유틸리티 함수 ---

def mean_pooling(model_output, attention_mask):
    """Mean Pooling 계산 함수"""
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

def get_embedding(text):
    """주어진 텍스트의 임베딩 벡터를 반환 (NumPy Array)"""
    try:
        encoded_input = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors='pt').to(device)
        with torch.no_grad():
            model_output = embedding_model(**encoded_input)
        embedding = mean_pooling(model_output, encoded_input['attention_mask'])
        return embedding.cpu().numpy().flatten() # NumPy 배열로 변환 후 flatten
    except Exception as e:
        print(f"Error getting embedding for text '{text[:50]}...': {e}")
        return None

# search_milvus 함수 정의
def search_milvus(query_vector, collection_names_list, top_k_total):
    """
    Milvus의 여러 컬렉션에서 관련성 높은 청크를 검색하고 결과를 병합합니다.
    (결과 처리 시 컬렉션별 필드 접근 분기 적용)
    """
    all_retrieved_chunks = []
    try:
        if not connections.has_connection("default"):
             connections.connect("default", host=MILVUS_HOST, port=MILVUS_PORT)

        for c_name in collection_names_list:
            # --- 컬렉션 확인, 로드, Output Fields 정의 (이전과 동일) ---
            print(f"\nSearching in collection: '{c_name}'...")
            # ... (collection 존재 확인, load 로직 동일) ...
            collection = Collection(c_name)
            try:
                print(f"  Ensuring collection '{c_name}' is loaded...")
                collection.load()
                utility.wait_for_loading_complete(c_name)
                print(f"  Collection '{c_name}' is ready for search.")
            except Exception as load_err:
                print(f"  Error loading collection '{c_name}': {load_err}")
                continue

            current_output_fields = []
            text_field_for_this_collection = ""
            if c_name == "celltrion_embeddings":
                 text_field_for_this_collection = "text"
                 current_output_fields = [text_field_for_this_collection]
            elif c_name == "news_embeddings":
                 text_field_for_this_collection = "chunk_text"
                 current_output_fields = [
                     text_field_for_this_collection, "original_article_id",
                     "chunk_seq_id", "title", "datetime", "summary", "url"
                 ]
            else:
                 print(f"  Warning: Unknown collection '{c_name}'. Skipping search.")
                 continue

            # --- 검색 실행 (이전과 동일) ---
            try:
                search_results = collection.search(
                    data=[query_vector.tolist()],
                    anns_field="embedding",
                    param=SEARCH_PARAMS,
                    limit=top_k_total,
                    output_fields=current_output_fields
                )

                # --- 결과 처리 (★★ 컬렉션별 필드 접근 분기 적용 ★★) ---
                if search_results and search_results[0]:
                    for hit in search_results[0]:
                        try:
                            hit_id = hit.id
                            hit_score = hit.distance
                            entity = hit.entity # entity 객체 가져오기

                            # 공통 필드 및 기본값으로 chunk_data 초기화
                            chunk_data = {
                                "collection": c_name,
                                "id": hit_id,
                                "score": hit_score,
                                # 텍스트 필드는 항상 요청되므로 getattr 사용 가능
                                "text": getattr(entity, text_field_for_this_collection, ""),
                                # source_type 기본값 설정 (스키마에 없으므로)
                                "source_type": c_name
                            }

                            # ★★★ 컬렉션에 따라 추가 필드 접근 ★★★
                            if c_name == "news_embeddings":
                                # news_embeddings 스키마에 있고 output_fields로 요청한 필드만 접근
                                chunk_data["title"] = getattr(entity, "title", "")
                                chunk_data["url"] = getattr(entity, "url", "")
                                chunk_data["original_article_id"] = getattr(entity, "original_article_id", None)
                                chunk_data["chunk_seq_id"] = getattr(entity, "chunk_seq_id", None)
                                chunk_data["datetime"] = getattr(entity, "datetime", "")
                                chunk_data["summary"] = getattr(entity, "summary", "")
                                # 만약 news 스키마에 source_type이 있다면 여기서 덮어쓰기
                                # chunk_data["source_type"] = getattr(entity, "source_type", c_name)

                            elif c_name == "celltrion_embeddings":
                                # celltrion_embeddings 스키마에는 추가 메타데이터 필드가 없음
                                # 따라서 여기서 접근할 필드 없음
                                pass
                                # 만약 celltrion 스키마에 source_type이 있다면 여기서 덮어쓰기
                                # chunk_data["source_type"] = getattr(entity, "source_type", c_name)

                            all_retrieved_chunks.append(chunk_data)

                        except Exception as process_err:
                            # 개별 hit 처리 중 예외 발생 시 로그 남기고 계속
                            print(f"  Warning: Error processing hit {getattr(hit, 'id', 'N/A')} in {c_name}: {process_err}")
                            continue # 다음 hit으로 넘어감

                    print(f"  Found {len(search_results[0])} results in '{c_name}'.")

            except Exception as search_err:
                print(f"  Error searching collection '{c_name}': {search_err}")

        # --- 결과 취합 후 정렬 및 제한 (이하 동일) ---
        if all_retrieved_chunks:
            all_retrieved_chunks.sort(key=lambda x: x['score'])
            print(f"\nTotal results from all collections: {len(all_retrieved_chunks)}")
            final_chunks = all_retrieved_chunks[:top_k_total]
            print(f"Returning top {len(final_chunks)} overall results.")
            return final_chunks
        else:
            return []

    except Exception as e:
        print(f"Error during Milvus search process: {e}")
        return []

def format_context(retrieved_chunks):
    """검색된 청크들을 LLM 프롬프트에 넣기 좋은 형태의 문자열로 변환"""
    context_str = ""
    for i, chunk in enumerate(retrieved_chunks):
        context_str += f"--- 문서 {i+1} (ID: {chunk['id']}, 출처: {chunk.get('title', 'N/A')}) ---\n"
        context_str += chunk.get('text', '') + "\n\n"
    return context_str.strip()

def ask_llm(query, context):
    """LLM에게 질문과 컨텍스트를 전달하고 답변을 받음"""
    prompt = f"""
{prompt_text}

[컨텍스트 정보]
{context if context else "제공된 컨텍스트 정보가 없습니다."}

[사용자 질문]
{query}

[답변]
"""

    try:
        client = OpenAI(api_key=OPENAI_API_KEY)
        response = client.chat.completions.create(
            model=LLM_MODEL,
            messages=[
                {"role": "system", "content": "You are a helpful assistant that answers questions based on provided context in Korean."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7, # 답변의 창의성 조절 (0에 가까울수록 결정적)
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error calling OpenAI API: {e}")
        return "OpenAI API 호출 중 오류가 발생했습니다."

Loading embedding model and tokenizer...
Using device for embedding: cpu


In [ ]:
from openai import OpenAI
import os

making_keyword_prompt = """
24년 4분기 셀트리온 기업 분석 보고서를 만들기 위해 RAG 파이프라인을 활용할 것입니다. 
벡터 DB인 Milvus에서 검색을 통해 상위 20개 청크를 가져올 것입니다.
보고서의 5번인 향후 전망을 만들 때 필요한 검색을 위한 키워드를 제출하세요.
결과를 출력할 때에는 키워드만 제출해주시고, 각 키워드들을 띄어쓰기로 구분하여 제출하세요.
"""

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
response = client.chat.completions.create(
    model=LLM_MODEL,
    messages=[
        {"role": "system", "content": "You are a helpful assistant that answers questions based on provided context in Korean."},
        {"role": "system", "content": prompt_text},
        {"role": "user", "content": making_keyword_prompt}
    ],
    temperature=0.7, # 답변의 창의성 조절 (0에 가까울수록 결정적)
)

print(response.choices[0].message.content.strip())

셀트리온 2024년 4분기 실적 분석 매출 비용 변동 요인 제품 판매 호조 부진 일회성 손익 공식 발표


In [8]:
keyword_embedding = get_embedding(response.choices[0].message.content.strip())

In [ ]:
user_query = "24년 4분기 셀트리온의 기업 분석 보고서 중 2번인 2024년 4분기 실적 분석에 대해 작성해주세요."

# 1. 키워드 쿼리 임베딩
print("Embedding your query...")
start_embed_time = time.time()
query_embedding = get_embedding(response.choices[0].message.content.strip())
embed_time = time.time() - start_embed_time
if query_embedding is None:
    print("쿼리 임베딩 중 오류가 발생했습니다.")
print(f"Keyword Query embedding done ({embed_time:.2f}s)")

# 2. Milvus 검색 (수정됨: 함수 호출 방식 변경)
print(f"Searching Milvus collections {COLLECTION_NAMES} for top {SEARCH_TOP_K} relevant chunks overall...")
start_search_time = time.time()
# 수정된 함수 호출: 컬렉션 이름 리스트와 최종 원하는 결과 개수 전달
retrieved_data = search_milvus(query_embedding, COLLECTION_NAMES, SEARCH_TOP_K)
search_time = time.time() - start_search_time

if not retrieved_data:
    print("Milvus에서 관련 정보를 찾지 못했습니다.")
    context_for_llm = ""
else:
    # 이제 retrieved_data는 여러 컬렉션의 결과를 포함하고 score 기준으로 정렬됨
    print(f"Found {len(retrieved_data)} relevant chunks overall ({search_time:.2f}s).")
    # (선택적) 검색 결과 미리보기 (출처 컬렉션 포함)
    # for i, chunk in enumerate(retrieved_data):
    #     print(f"  Result {i+1}: Score={chunk['score']:.4f}, Collection='{chunk['collection']}', Text={chunk['text'][:80]}...")
    context_for_llm = format_context(retrieved_data) # format_context 함수는 그대로 사용 가능

print(retrieved_data)

# 3. LLM에게 질문/답변 생성
print("Asking LLM...")
start_llm_time = time.time()
answer = ask_llm(user_query, context_for_llm)
llm_time = time.time() - start_llm_time
print(f"LLM response received ({llm_time:.2f}s).")

# 4. 답변 출력
print("\n[답변]")
print(answer)


Embedding your query...
Keyword Query embedding done (0.07s)
Searching Milvus collections ['celltrion_embeddings', 'news_embeddings'] for top 20 relevant chunks overall...

Searching in collection: 'celltrion_embeddings'...
  Ensuring collection 'celltrion_embeddings' is loaded...
  Collection 'celltrion_embeddings' is ready for search.
  Found 20 results in 'celltrion_embeddings'.

Searching in collection: 'news_embeddings'...
  Ensuring collection 'news_embeddings' is loaded...
  Collection 'news_embeddings' is ready for search.
  Found 20 results in 'news_embeddings'.

Total results from all collections: 40
Returning top 20 overall results.
Found 20 relevant chunks overall (0.30s).
[{'collection': 'news_embeddings', 'id': 456911237736039911, 'score': 125.45536041259766, 'text': '장민환 iM증권 연구원은 보고서를 통해 "마진율이 높은 후속제품의 성장세가 고무적으로 내년 신규 출시되는 5건의 품목이 추가되면 후속 제품 매출 비중은 약 48%로 증가할 전망"이라며 "짐펜트라의 불확실성이 존재하나 그 외 제품의 실적 견인과 위탁생산(CMO) 사업 진입, R&D 성과 및 주주환원이 공존하고 단기적으로는 기존 제품 점유율 유지와 신규 품목 매출 성장에 

In [ ]:
from openai import OpenAI

making_keyword_prompt = """
24년 4분기 셀트리온 기업 분석 보고서를 만들기 위해 RAG 파이프라인을 활용할 것입니다. 
벡터 DB인 Milvus에서 검색을 통해 상위 20개 청크를 가져올 것입니다.
보고서의 5번인 향후 전망을 만들 때 필요한 검색을 위한 키워드를 제출하세요.
결과를 출력할 때에는 키워드만 제출해주시고, 각 키워드들을 띄어쓰기로 구분하여 제출하세요.
"""

client = OpenAI(api_key=OPENAI_API_KEY)
response = client.chat.completions.create(
    model=LLM_MODEL,
    messages=[
        {"role": "system", "content": "You are a helpful assistant that answers questions based on provided context in Korean."},
        {"role": "system", "content": prompt_text},
        {"role": "user", "content": making_keyword_prompt}
    ],
    temperature=0.7, # 답변의 창의성 조절 (0에 가까울수록 결정적)
)

print(response.choices[0].message.content.strip())

셀트리온 4분기 실적 분석 주요 사업 및 제품 동향 신약 개발 R&D 제품 출시 허가 현황 판매 동향 생산 업데이트 CMO 사업 합병 시너지


In [ ]:
user_query = "24년 4분기 셀트리온의 기업 분석 보고서 중 3번인 주요 사업 및 제품 동향향에 대해 작성해주세요."

# 1. 키워드 쿼리 임베딩
print("Embedding your query...")
start_embed_time = time.time()
query_embedding = get_embedding(response.choices[0].message.content.strip())
embed_time = time.time() - start_embed_time
if query_embedding is None:
    print("쿼리 임베딩 중 오류가 발생했습니다.")
print(f"Keyword Query embedding done ({embed_time:.2f}s)")

# 2. Milvus 검색 (수정됨: 함수 호출 방식 변경)
print(f"Searching Milvus collections {COLLECTION_NAMES} for top {SEARCH_TOP_K} relevant chunks overall...")
start_search_time = time.time()
# 수정된 함수 호출: 컬렉션 이름 리스트와 최종 원하는 결과 개수 전달
retrieved_data = search_milvus(query_embedding, COLLECTION_NAMES, SEARCH_TOP_K)
search_time = time.time() - start_search_time

if not retrieved_data:
    print("Milvus에서 관련 정보를 찾지 못했습니다.")
    context_for_llm = ""
else:
    # 이제 retrieved_data는 여러 컬렉션의 결과를 포함하고 score 기준으로 정렬됨
    print(f"Found {len(retrieved_data)} relevant chunks overall ({search_time:.2f}s).")
    # (선택적) 검색 결과 미리보기 (출처 컬렉션 포함)
    # for i, chunk in enumerate(retrieved_data):
    #     print(f"  Result {i+1}: Score={chunk['score']:.4f}, Collection='{chunk['collection']}', Text={chunk['text'][:80]}...")
    context_for_llm = format_context(retrieved_data) # format_context 함수는 그대로 사용 가능

print(retrieved_data)

# 3. LLM에게 질문/답변 생성
print("Asking LLM...")
start_llm_time = time.time()
answer = ask_llm(user_query, context_for_llm)
llm_time = time.time() - start_llm_time
print(f"LLM response received ({llm_time:.2f}s).")

# 4. 답변 출력
print("\n[답변]")
print(answer)

Embedding your query...
Keyword Query embedding done (0.08s)
Searching Milvus collections ['celltrion_embeddings', 'news_embeddings'] for top 20 relevant chunks overall...

Searching in collection: 'celltrion_embeddings'...
  Ensuring collection 'celltrion_embeddings' is loaded...
  Collection 'celltrion_embeddings' is ready for search.
  Found 20 results in 'celltrion_embeddings'.

Searching in collection: 'news_embeddings'...
  Ensuring collection 'news_embeddings' is loaded...
  Collection 'news_embeddings' is ready for search.
  Found 20 results in 'news_embeddings'.

Total results from all collections: 40
Returning top 20 overall results.
Found 20 relevant chunks overall (0.04s).
[{'collection': 'celltrion_embeddings', 'id': 456911237736039789, 'score': 85.73736572265625, 'text': '유럽 임상 3상 시험계획 Part1, Part2 동시 승인에 대한 공시인 점 참고 부탁드립니다. 본 공시 내용은 향후 당사의 보도자료 및 IR 자료로 사용될 예정입니다. 2024 - 08 - 23 투자판단 관련 주요경영사항 ( CTP51 ( 키트루다 바이오시밀러 ) 유럽 관련공시 임상 3상 시험계획 신청 ( Part 1 & 2 ) )', 'source_type'

In [ ]:
from openai import OpenAI

making_keyword_prompt = """
24년 4분기 셀트리온 기업 분석 보고서를 만들기 위해 RAG 파이프라인을 활용할 것입니다. 
벡터 DB인 Milvus에서 검색을 통해 상위 20개 청크를 가져올 것입니다.
보고서의 5번인 향후 전망을 만들 때 필요한 검색을 위한 키워드를 제출하세요.
결과를 출력할 때에는 키워드만 제출해주시고, 각 키워드들을 띄어쓰기로 구분하여 제출하세요.
"""

client = OpenAI(api_key=OPENAI_API_KEY)
response = client.chat.completions.create(
    model=LLM_MODEL,
    messages=[
        {"role": "system", "content": "You are a helpful assistant that answers questions based on provided context in Korean."},
        {"role": "system", "content": prompt_text},
        {"role": "user", "content": making_keyword_prompt}
    ],
    temperature=0.7, # 답변의 창의성 조절 (0에 가까울수록 결정적)
)

print(response.choices[0].message.content.strip())

셀트리온 시장 환경 전략 방향 미국 유럽 신제품 출시 허가 신청 마케팅 활동 시장 경쟁 경영 전략 투자 계획 파트너십 체결


In [13]:
user_query = "24년 4분기 셀트리온의 기업 분석 보고서 중 4번인 시장 환경 및 전략 방향에 대해 작성해주세요."

# 1. 키워드 쿼리 임베딩
print("Embedding your query...")
start_embed_time = time.time()
query_embedding = get_embedding(response.choices[0].message.content.strip())
embed_time = time.time() - start_embed_time
if query_embedding is None:
    print("쿼리 임베딩 중 오류가 발생했습니다.")
print(f"Keyword Query embedding done ({embed_time:.2f}s)")

# 2. Milvus 검색 (수정됨: 함수 호출 방식 변경)
print(f"Searching Milvus collections {COLLECTION_NAMES} for top {SEARCH_TOP_K} relevant chunks overall...")
start_search_time = time.time()
# 수정된 함수 호출: 컬렉션 이름 리스트와 최종 원하는 결과 개수 전달
retrieved_data = search_milvus(query_embedding, COLLECTION_NAMES, SEARCH_TOP_K)
search_time = time.time() - start_search_time

if not retrieved_data:
    print("Milvus에서 관련 정보를 찾지 못했습니다.")
    context_for_llm = ""
else:
    # 이제 retrieved_data는 여러 컬렉션의 결과를 포함하고 score 기준으로 정렬됨
    print(f"Found {len(retrieved_data)} relevant chunks overall ({search_time:.2f}s).")
    # (선택적) 검색 결과 미리보기 (출처 컬렉션 포함)
    # for i, chunk in enumerate(retrieved_data):
    #     print(f"  Result {i+1}: Score={chunk['score']:.4f}, Collection='{chunk['collection']}', Text={chunk['text'][:80]}...")
    context_for_llm = format_context(retrieved_data) # format_context 함수는 그대로 사용 가능

print(retrieved_data)

# 3. LLM에게 질문/답변 생성
print("Asking LLM...")
start_llm_time = time.time()
answer = ask_llm(user_query, context_for_llm)
llm_time = time.time() - start_llm_time
print(f"LLM response received ({llm_time:.2f}s).")

# 4. 답변 출력
print("\n[답변]")
print(answer)

Embedding your query...
Keyword Query embedding done (0.12s)
Searching Milvus collections ['celltrion_embeddings', 'news_embeddings'] for top 20 relevant chunks overall...

Searching in collection: 'celltrion_embeddings'...
  Ensuring collection 'celltrion_embeddings' is loaded...
  Collection 'celltrion_embeddings' is ready for search.
  Found 20 results in 'celltrion_embeddings'.

Searching in collection: 'news_embeddings'...
  Ensuring collection 'news_embeddings' is loaded...
  Collection 'news_embeddings' is ready for search.
  Found 20 results in 'news_embeddings'.

Total results from all collections: 40
Returning top 20 overall results.
Found 20 relevant chunks overall (0.04s).
[{'collection': 'celltrion_embeddings', 'id': 456911237736039789, 'score': 130.7427215576172, 'text': '유럽 임상 3상 시험계획 Part1, Part2 동시 승인에 대한 공시인 점 참고 부탁드립니다. 본 공시 내용은 향후 당사의 보도자료 및 IR 자료로 사용될 예정입니다. 2024 - 08 - 23 투자판단 관련 주요경영사항 ( CTP51 ( 키트루다 바이오시밀러 ) 유럽 관련공시 임상 3상 시험계획 신청 ( Part 1 & 2 ) )', 'source_type'

In [ ]:
from openai import OpenAI

making_keyword_prompt = """
24년 4분기 셀트리온 기업 분석 보고서를 만들기 위해 RAG 파이프라인을 활용할 것입니다. 
벡터 DB인 Milvus에서 검색을 통해 상위 20개 청크를 가져올 것입니다.
보고서의 5번인 향후 전망을 만들 때 필요한 검색을 위한 키워드를 제출하세요.
결과를 출력할 때에는 키워드만 제출해주시고, 각 키워드들을 띄어쓰기로 구분하여 제출하세요.
"""

client = OpenAI(api_key=OPENAI_API_KEY)
response = client.chat.completions.create(
    model=LLM_MODEL,
    messages=[
        {"role": "system", "content": "You are a helpful assistant that answers questions based on provided context in Korean."},
        {"role": "system", "content": prompt_text},
        {"role": "user", "content": making_keyword_prompt}
    ],
    temperature=0.7, # 답변의 창의성 조절 (0에 가까울수록 결정적)
)

print(response.choices[0].message.content.strip())

2025년 사업 계획 신제품 출시 목표 성장 전략


In [ ]:
user_query = "24년 4분기 셀트리온의 기업 분석 보고서 중 5번인 향후 전망에 대해 작성해주세요."

# 1. 키워드 쿼리 임베딩
print("Embedding your query...")
start_embed_time = time.time()
query_embedding = get_embedding(response.choices[0].message.content.strip())
embed_time = time.time() - start_embed_time
if query_embedding is None:
    print("쿼리 임베딩 중 오류가 발생했습니다.")
print(f"Keyword Query embedding done ({embed_time:.2f}s)")

# 2. Milvus 검색 (수정됨: 함수 호출 방식 변경)
print(f"Searching Milvus collections {COLLECTION_NAMES} for top {SEARCH_TOP_K} relevant chunks overall...")
start_search_time = time.time()
# 수정된 함수 호출: 컬렉션 이름 리스트와 최종 원하는 결과 개수 전달
retrieved_data = search_milvus(query_embedding, COLLECTION_NAMES, SEARCH_TOP_K)
search_time = time.time() - start_search_time

if not retrieved_data:
    print("Milvus에서 관련 정보를 찾지 못했습니다.")
    context_for_llm = ""
else:
    # 이제 retrieved_data는 여러 컬렉션의 결과를 포함하고 score 기준으로 정렬됨
    print(f"Found {len(retrieved_data)} relevant chunks overall ({search_time:.2f}s).")
    # (선택적) 검색 결과 미리보기 (출처 컬렉션 포함)
    # for i, chunk in enumerate(retrieved_data):
    #     print(f"  Result {i+1}: Score={chunk['score']:.4f}, Collection='{chunk['collection']}', Text={chunk['text'][:80]}...")
    context_for_llm = format_context(retrieved_data) # format_context 함수는 그대로 사용 가능

print(retrieved_data)

# 3. LLM에게 질문/답변 생성
print("Asking LLM...")
start_llm_time = time.time()
answer = ask_llm(user_query, context_for_llm)
llm_time = time.time() - start_llm_time
print(f"LLM response received ({llm_time:.2f}s).")

# 4. 답변 출력
print("\n[답변]")
print(answer)

Embedding your query...
Keyword Query embedding done (0.25s)
Searching Milvus collections ['celltrion_embeddings', 'news_embeddings'] for top 20 relevant chunks overall...

Searching in collection: 'celltrion_embeddings'...
  Ensuring collection 'celltrion_embeddings' is loaded...
  Collection 'celltrion_embeddings' is ready for search.
  Found 20 results in 'celltrion_embeddings'.

Searching in collection: 'news_embeddings'...
  Ensuring collection 'news_embeddings' is loaded...
  Collection 'news_embeddings' is ready for search.
  Found 20 results in 'news_embeddings'.

Total results from all collections: 40
Returning top 20 overall results.
Found 20 relevant chunks overall (0.13s).
[{'collection': 'news_embeddings', 'id': 456911237736039816, 'score': 179.134521484375, 'text': '특히 고부가가치 서비스 중심으로 운영해 1만L 규모당 최소 매출 1000억 원 이상을 기록하도록 사업을 꾸려갈 예정이며 목표 영업이익률은 30% 중반으로 잡았다. vrdw88@fnnews.com 강중모 기자', 'source_type': 'news_embeddings', 'title': '셀트리온 "CDMO로 제2의 도약"', 'url': 'https://n.news.na

In [ ]:
from openai import OpenAI
import os

making_keyword_prompt = """
24년 4분기 셀트리온 기업 분석 보고서를 만들기 위해 RAG 파이프라인을 활용할 것입니다. 
벡터 DB인 Milvus에서 검색을 통해 상위 20개 청크를 가져올 것입니다.
보고서의 6번인 기타 참고사항항을 만들 때 필요한 검색을 위한 키워드를 제출하세요.
결과를 출력할 때에는 키워드만 제출해주시고, 각 키워드들을 띄어쓰기로 구분하여 제출하세요.
"""

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
response = client.chat.completions.create(
    model=LLM_MODEL,
    messages=[
        {"role": "system", "content": "You are a helpful assistant that answers questions based on provided context in Korean."},
        {"role": "system", "content": prompt_text},
        {"role": "user", "content": making_keyword_prompt}
    ],
    temperature=0.7, # 답변의 창의성 조절 (0에 가까울수록 결정적)
)

print(response.choices[0].message.content.strip())

리스크 기회 주요 정보 잠재적 변화 시장 동향 경쟁사 분석 재무 안정성 규제 환경 투자자 관심 파트너십 동향 합병 시너지


In [18]:
user_query = "24년 4분기 셀트리온의 기업 분석 보고서 중 6번인 기타 참고사항에 대해 작성해주세요."

# 1. 키워드 쿼리 임베딩
print("Embedding your query...")
start_embed_time = time.time()
query_embedding = get_embedding(response.choices[0].message.content.strip())
embed_time = time.time() - start_embed_time
if query_embedding is None:
    print("쿼리 임베딩 중 오류가 발생했습니다.")
print(f"Keyword Query embedding done ({embed_time:.2f}s)")

# 2. Milvus 검색 (수정됨: 함수 호출 방식 변경)
print(f"Searching Milvus collections {COLLECTION_NAMES} for top {SEARCH_TOP_K} relevant chunks overall...")
start_search_time = time.time()
# 수정된 함수 호출: 컬렉션 이름 리스트와 최종 원하는 결과 개수 전달
retrieved_data = search_milvus(query_embedding, COLLECTION_NAMES, SEARCH_TOP_K)
search_time = time.time() - start_search_time

if not retrieved_data:
    print("Milvus에서 관련 정보를 찾지 못했습니다.")
    context_for_llm = ""
else:
    # 이제 retrieved_data는 여러 컬렉션의 결과를 포함하고 score 기준으로 정렬됨
    print(f"Found {len(retrieved_data)} relevant chunks overall ({search_time:.2f}s).")
    # (선택적) 검색 결과 미리보기 (출처 컬렉션 포함)
    # for i, chunk in enumerate(retrieved_data):
    #     print(f"  Result {i+1}: Score={chunk['score']:.4f}, Collection='{chunk['collection']}', Text={chunk['text'][:80]}...")
    context_for_llm = format_context(retrieved_data) # format_context 함수는 그대로 사용 가능

print(retrieved_data)

# 3. LLM에게 질문/답변 생성
print("Asking LLM...")
start_llm_time = time.time()
answer = ask_llm(user_query, context_for_llm)
llm_time = time.time() - start_llm_time
print(f"LLM response received ({llm_time:.2f}s).")

# 4. 답변 출력
print("\n[답변]")
print(answer)

Embedding your query...
Keyword Query embedding done (0.08s)
Searching Milvus collections ['celltrion_embeddings', 'news_embeddings'] for top 20 relevant chunks overall...

Searching in collection: 'celltrion_embeddings'...
  Ensuring collection 'celltrion_embeddings' is loaded...
  Collection 'celltrion_embeddings' is ready for search.
  Found 20 results in 'celltrion_embeddings'.

Searching in collection: 'news_embeddings'...
  Ensuring collection 'news_embeddings' is loaded...
  Collection 'news_embeddings' is ready for search.
  Found 20 results in 'news_embeddings'.

Total results from all collections: 40
Returning top 20 overall results.
Found 20 relevant chunks overall (0.08s).
[{'collection': 'celltrion_embeddings', 'id': 456911237736039793, 'score': 194.2028045654297, 'text': '내용은 향후 당사의 보도자료 및 IR 자료로 사용될 예정입니다. 2024 - 03 - 08 투자판단 관련 주요경영사항 ( CT - P41 ( 프롤리아 & 엑스지바 바이오 관련공시 시밀러 ) 유럽 품목허가 신청 )', 'source_type': 'celltrion_embeddings'}, {'collection': 'celltrion_embeddings', 'id

In [20]:
from openai import OpenAI

making_summary_prompt = """
### 2024년 4분기 실적 분석

셀트리온의 2024년 4분기 실적은 여러 요인에 의해 영향을 받을 것으로 예상된다. 다음은 4분기 실적의 주요 변동 요인에 대한 분석이다.

1. **매출 성장 요인**:
    - 셀트리온은 기존 제품의 시장 점유율 확대와 신규 입찰 수주가 이어지고 있으며, 특히 후속 바이오시밀러 제품의 성장이 고무적이다. 장민환 iM증권 연구원에 따르면, 신규 출시되는 5건의 품목이 추가되면 후속 제품 매출 비중이 약 48%로 증가할 것으로 전망된다. 이는 매출 증가에 긍정적인 영향을 미칠 것으로 보인다.
    - 짐펜트라의 경우, 미국에서의 처방약급여관리업체(PBM) 커버리지 확보가 진행되면서 매출이 본격적으로 성장할 것으로 기대된다. 현재 3대 PBM의 처방집에 모두 등재되어 있어, 향후 성과에 대한 기대가 높아지고 있다.
2. **매출 부진 요인**:
    - 짐펜트라의 2·3분기 매출이 100억원을 밑돌며 시장 기대에 미치지 못하고 있는 상황이다. 이는 낮은 브랜드 인지도와 주요 처방약급여관리업체(PBM) 등재의 지연이 원인으로 분석되고 있다. 이런 부정적인 요소는 4분기 실적에도 영향을 미칠 수 있다.
3. **비용 요인**:
    - 셀트리온은 R&D 및 생산 역량 강화를 위한 지속적인 투자 외에도, 위탁생산(CMO) 사업 진입 등을 통해 비용 구조 개선을 모색하고 있다. 그러나 이러한 비용 증가가 단기적으로는 실적에 부정적인 영향을 미칠 가능성도 존재한다.
4. **일회성 손익**:
    - 4분기 동안 특정 일회성 손익이 발생할 가능성이 있으며, 이는 전체 실적에 영향을 줄 수 있다. 예를 들어, CMO 사업 관련 계약 체결 및 매출 발생 등이 일회성으로 나타날 수 있다.

이러한 요인들은 셀트리온의 2024년 4분기 실적에 복합적으로 작용할 것으로 예상되며, 특히 신규 제품의 판매 성과와 기존 제품의 점유율 유지가 중요한 변수가 될 것이다.

### 3. 주요 사업 및 제품 동향 (Business & Product Developments)

### 제품 관련 소식

셀트리온은 현재 9개의 바이오시밀러 제품 포트폴리오를 보유하고 있으며, 최근 '옴리클로', '아이덴젤트', '스테키마'와 같은 신규 제품에 대한 허가를 획득하였습니다. 이 외에도 류마티스 관절염 치료제 '악템라', 골다공증 치료제 '프롤리아', 다발성경화증 치료제 '오크레부스' 등 후속 바이오시밀러 제품의 허가 절차가 진행되고 있습니다. 이러한 제품들은 셀트리온의 글로벌 시장 점유율 확대에 기여할 것으로 기대됩니다.

특히, 셀트리온은 자가면역질환 치료 신약 '짐펜트라'의 미국 시장 진출 이후, 처방약급여관리업체(PBM)와의 커버리지를 확대하여 성과를 내고 있으며, 이 제품은 향후 매출 성장의 중요한 동력으로 기대되고 있습니다. 짐펜트라는 3대 PBM 모두의 처방 목록에 등재되어 있어 시장에서의 입지를 강화하고 있습니다.

### R&D 및 파이프라인

셀트리온은 연구개발에 연간 매출액의 약 20%를 투자하고 있으며, 바이오시밀러 파이프라인 확대와 항체의약품 신약 개발을 지속적으로 추진하고 있습니다. 최근에는 CT-P47 (악템라 바이오시밀러)의 글로벌 임상 3상 결과에서 오리지널 의약품 대비 동등성 및 유사성을 확인하였으며, 이 결과를 바탕으로 해외 주요 국가에서 허가 신청을 가속화할 계획입니다. 또한, CT-P51 (키트루다 바이오시밀러)의 유럽 임상 3상 시험계획이 동시 승인됨에 따라, 향후 시장 진출 기대가 높아지고 있습니다.

### CMO 사업

셀트리온은 기존의 CMO 사업을 한층 강화하기 위해 새로운 의약품 위탁개발생산(CDMO) 자회사를 설립하였습니다. 이 자회사는 신약 후보물질 선별부터 상업 생산까지 전 주기 서비스를 제공할 예정이며, 셀트리온의 항체 개발 및 생산 노하우를 활용하여 경쟁력을 높일 계획입니다. 이는 셀트리온의 CDMO 사업 진출을 본격화하는 중요한 단계로, 향후 글로벌 바이오의약품 수요 확대에 대응할 수 있는 기반을 마련할 것입니다.

### 기타 사업

셀트리온은 최근 3개국(한국, 홍콩, 대만)에서 OTC(일반의약품) 영업 양도 계약을 체결한 바 있습니다. 이를 통해 핵심사업에 집중하고 사업 구조를 재편하여 기업 가치를 제고하는 목표를 가지고 있습니다. 이와 같은 전략은 셀트리온의 재무구조 개선에도 긍정적인 영향을 미칠 것으로 기대됩니다.

이와 같은 주요 사업 및 제품 동향은 셀트리온의 지속적인 성장과 글로벌 시장에서의 경쟁력 강화를 위한 중요한 요소로 작용하고 있습니다.

## 4. 시장 환경 및 전략 방향 (Market Environment & Strategy)

### 주요 시장 활동

셀트리온은 2024년 4분기 동안 주요 시장인 미국과 유럽에서 활발한 활동을 전개하고 있습니다. 특히, 독일 시장에서 램시마 제품군(IV∙SC)의 점유율이 71%에 달하며, 이는 셀트리온의 제품 경쟁력을 강화하는 데 중요한 역할을 하고 있습니다. 스테키마의 출시를 통해 셀트리온은 항체 바이오 의약품 분야에서의 입지를 더욱 확고히 하고 있으며, 자가면역질환 포트폴리오를 확장하여 의료진과 환자 선택권을 높이고 있습니다.

또한, 호주 시장에서도 램시마SC의 판매가 증가하고 있으며, 현지 법인이 주요 이해관계자와의 네트워크를 강화하고 임상 데이터를 알리면서 처방 선호도를 높이고 있습니다. 이러한 현지화 전략은 셀트리온의 글로벌 시장 점유율 확대에 긍정적인 영향을 미치고 있습니다.

### 회사의 공식 전략

셀트리온은 2024년 12월에 유럽 30개국에서의 판매 승인 권고를 획득한 바이오시밀러 제품을 통해 글로벌 시장에서의 경쟁력을 강화하고 있습니다. 회사는 2030년까지 22개 제품 라인업 구축을 목표로 하며, 바이오시밀러 시장의 높은 성장세를 활용하여 지속적으로 시장 점유율을 확대할 계획입니다.

셀트리온은 CDMO(의약품 위탁개발생산) 사업 진출을 통해 신규 수익원을 창출하고, 의약품 개발 전주기 서비스를 제공하는 신규 자회사를 설립하였습니다. 이 자회사는 신약 후보물질 선별부터 상업 생산까지 모든 단계를 지원하며, 경쟁력 있는 생산성과 원가 절감에 중점을 두고 운영될 것입니다.

또한, 셀트리온은 베트남 시장에 진출하여 유플라이마, 베그젤마, 옴리클로 등의 바이오시밀러 제품을 출시할 예정이며, 현지 네트워크 구축과 영업 활동을 통해 시장 점유율을 높이는 전략을 추진하고 있습니다.

이와 같은 전략적 방향은 셀트리온이 글로벌 바이오의약품 시장에서 지속 가능한 성장을 이루는 데 중요한 요소로 작용할 것입니다.

### 5. 향후 전망 (공식 발표 기반)

셀트리온은 향후 2025년 사업 계획과 목표에 대해 구체적인 비전을 제시하고 있습니다. 회사는 2025년까지 바이오시밀러 사업 부문에서 11개 제품의 허가를 획득하고, 2030년까지 22개 제품의 포트폴리오를 확립하여 시장 지배력을 강화할 계획입니다. 이는 자가면역질환 치료제뿐만 아니라 천식, 두드러기, 안과, 대사성 골질환 등 다양한 치료 영역으로의 확장을 포함합니다. 이를 통해 다제품 전략을 통해 처방약급여관리업체(PBM)와의 협상력을 강화하고 판매 효율성을 높이겠다는 전략을 세우고 있습니다.

서정진 회장은 자가면역질환 치료제 ‘짐펜트라’가 미국 시장에서 점유율을 확대하고 있으며, 내년에는 매출 5조원 달성을 목표로 하고 있다고 밝혔습니다. 올해 목표 매출인 2500억원은 충분히 달성 가능할 것으로 전망하고 있으며, 다른 바이오시밀러 제품들도 주요 시장에서 점유율을 꾸준히 확대하고 있다는 자신감을 드러냈습니다.

또한, 셀트리온은 CDMO(위탁개발생산) 사업을 본격적으로 추진하고 있으며, 연내 자회사를 설립하여 CDMO 사업의 경쟁력을 높이는 데 집중할 계획입니다. 이 사업은 2028년부터 매출이 발생할 것으로 예상하고 있으며, 항체, 이중항체, 펩타이드 등 다양한 서비스를 제공하여 글로벌 시장에서의 입지를 강화하겠다는 목표를 가지고 있습니다.

이와 함께 셀트리온은 신규 제조소 확보와 관련하여 결정은 연내 마무리 짓겠다고 밝혔으며, 이를 통해 글로벌 톱티어급 생산 능력을 구축하고 의약품 공급 사이클의 모든 단계에서 고객의 요구에 맞춤형 서비스를 제공할 수 있을 것으로 기대하고 있습니다.

향후 실적에 영향을 미칠 수 있는 주요 요인으로는 신제품 성과, 진행 중인 R&D의 중요성, 시장 환경의 변화 등을 포함할 수 있으며, 회사의 지속적인 성장과 발전을 위한 노력은 기업의 가치를 높이는 데 기여할 것으로 전망됩니다.

## 6. 기타 참고사항

1. **리스크 요인**:
    - 셀트리온의 사업 전략 및 성과는 글로벌 시장의 경쟁 환경, 규제 변화, 그리고 경제적 요인에 따라 영향을 받을 수 있습니다. 특히, 바이오시밀러 제품의 경우, 경쟁사가 출시하는 유사 제품의 개발 속도와 시장 반응이 중요한 변수로 작용할 수 있습니다.
    - 또한, 임상시험 및 제품 허가 과정에서 발생할 수 있는 지연이나 실패는 재무적 측면에서 부정적인 영향을 미칠 수 있습니다.
2. **기회 요인**:
    - 셀트리온은 바이오시밀러와 신약 개발 부문에서 지속적인 연구개발 투자로 새로운 치료 옵션을 제공할 수 있는 기회를 갖고 있습니다. 특히, 다중항체 플랫폼 기술 개발과 같은 혁신적인 접근 방식은 향후 시장에서의 경쟁력을 강화할 가능성이 있습니다.
    - 글로벌 시장에서의 판매 확대 및 파트너십 체결을 통해 매출 성장을 도모할 수 있는 잠재적인 기회가 존재합니다.
3. **전략적 제휴 및 파트너십**:
    - 셀트리온은 여러 글로벌 제약사와의 협력 및 라이선스 계약을 통해 개발 비용을 분담하고 새로운 시장에 진입할 수 있는 기회를 모색하고 있습니다.
    - 이러한 전략적 제휴는 기술력 향상과 시장 점유율 확대에 기여할 수 있습니다.
4. **주주 가치 제고**:
    - 셀트리온은 2024년 내 자기주식 매입을 검토 중이며, 이를 통해 주주 가치를 증대시킬 계획을 세우고 있습니다.
5. **기타 중요 사항**:
    - 셀트리온의 연구개발 투자 비율은 매출액의 약 20%에 달하며, 이는 바이오시밀러 및 신약 개발에 대한 의지를 반영합니다.
    - 회사는 향후 중장기적인 사업 계획 및 경영 전략을 지속적으로 업데이트할 예정이며, 이는 투자자들에게 중요한 정보로 작용할 수 있습니다.

위 보고서는 현재 당신이 만든 2번부터 6번까지의 보고서 내용입니다.
이를 참고하여 보고서의 1번인 보고서 요약을 작성해주세요
"""

client = OpenAI(api_key=OPENAI_API_KEY)
response = client.chat.completions.create(
    model=LLM_MODEL,
    messages=[
        {"role": "system", "content": "You are a helpful assistant that answers questions based on provided context in Korean."},
        {"role": "system", "content": prompt_text},
        {"role": "user", "content": making_summary_prompt}
    ],
    temperature=0.7, # 답변의 창의성 조절 (0에 가까울수록 결정적)
)

print(response.choices[0].message.content.strip())

### 1. 보고서 요약 (Executive Summary)

셀트리온의 2024년 4분기 실적은 다양한 요인에 의해 영향을 받을 것으로 예상되며, 매출 성장 요인으로는 기존 제품의 시장 점유율 확대와 신규 제품의 출시가 있다. 특히 후속 바이오시밀러 제품의 성장이 두드러지며, 짐펜트라의 경우 미국에서의 처방약급여관리업체(PBM) 커버리지 확보가 긍정적인 요소로 작용할 것으로 보인다. 그러나, 짐펜트라의 초기 매출 부진과 높은 연구개발(R&D) 및 생산 비용 증가가 실적에 부정적인 영향을 미칠 수 있다.

셀트리온은 현재 9개의 바이오시밀러 제품 포트폴리오를 보유하고 있으며, '옴리클로', '아이덴젤트', '스테키마' 등 신규 제품의 허가를 획득하여 글로벌 시장 점유율 확대를 기대하고 있다. R&D 분야에서는 CT-P47 및 CT-P51의 임상 결과가 긍정적으로 나타나 향후 시장 진출 기대가 커지고 있다. 또한, CMO 사업을 통해 신규 수익원을 창출하고 있으며, OTC 영업 양도를 통해 사업 구조를 재편하고 있다.

셀트리온은 미국 및 유럽 시장에서의 활동을 강화하고 있으며, 2030년까지 22개 제품 라인업 구축을 목표로 하고 있다. 향후 2025년에는 11개 바이오시밀러 제품의 허가를 계획하고 있으며, 자가면역질환 치료제 '짐펜트라'와 CDMO 사업의 확장을 통해 지속 가능한 성장을 추구하고 있다. 

향후 실적에 영향을 미칠 변수로는 신제품 성과, R&D 진행 상황, 시장 환경 변화 등이 있으며, 셀트리온은 지속적인 연구개발 투자와 글로벌 파트너십을 통해 기업 가치를 높이는 데 주력하고 있다.
